# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [127]:
# Load the libraries as required.
%load_ext dotenv
%dotenv 

import os
import sys
sys.path.append(os.getenv('SRC_DIR'))
from logger import get_logger
_logs = get_logger(__name__)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn import set_config
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor


import dask.dataframe as dd
import pandas as pd
import numpy as np
import os
from glob import glob
import pickle

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [25]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [26]:
x = fires_dt.drop('area', axis=1)
y = fires_dt['area']

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
#seperate numerical and categorical data from the features data frame
#numerical_data = x[['coord_x','coord_y','ffmc','dmc','dc','isi','temp','rh','wind','rain']]
#categorical_data = x[['month', 'day']]

In [27]:
numerical_data = ['coord_x','coord_y','ffmc','dmc','dc','isi','temp','rh','wind','rain']
categorical_data = ['month','day']

In [28]:
# Create a StandardScaler object for the numerical values

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# Fit the StandardScaler object with the numerical data
#std_scaler.fit(numerical_data)

In [29]:
# Create a OneHoteEncoder object for the categorical values
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()

#Fit the OneHotEncoder object with the categorical data
#onehot.fit(categorical_data)

In [30]:
preproc_1 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), numerical_data),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), categorical_data), 
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [31]:
preproc_2 = ColumnTransformer(
    transformers=[
        ('transform', PowerTransformer(method='yeo-johnson'), numerical_data),
        ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist'), categorical_data), 
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [32]:
# Pipeline A = preproc1 + baseline
pipeline_a = Pipeline([   
    ('preprocessing', preproc_1), 
    ('regressor', LinearRegression())
])

In [33]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline([   
    ('preprocessing', preproc_2), 
    ('regressor', LinearRegression())
])

In [34]:
# Pipeline C = preproc1 + advanced model
pipeline_c = Pipeline([   
    ('preprocessing', preproc_1), 
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', LinearRegression())
])

In [35]:
# Pipeline D = preproc2 + advanced model
pipeline_d = Pipeline([   
    ('preprocessing', preproc_2), 
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', LinearRegression())
])
    

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

scoring = ['neg_mean_squared_error', 'r2']

In [37]:
res_simple_dict_a = cross_validate(pipeline_a, x_train, y_train, cv = 5, scoring = scoring)
res_simple_a = pd.DataFrame(res_simple_dict_a).assign(experiment = 1)
res_simple_a

,fit_time,score_time,test_neg_mean_squared_error,test_r2,experiment
0,0.006913,0.010290,-1662.742068,-0.019267,1
1,0.006053,0.002994,-383.302544,-0.771829,1
2,0.009558,0.002916,-1113.391778,-0.545676,1
3,0.005190,0.002009,-7270.832903,-0.009167,1
4,0.007034,0.004095,-723.353265,-0.232067,1


In [ ]:
res_simple_a.mean()

In [38]:
res_simple_dict_b = cross_validate(pipeline_b, x_train, y_train, cv = 5, scoring = scoring)
res_simple_b = pd.DataFrame(res_simple_dict_b).assign(experiment = 2)
res_simple_b

,fit_time,score_time,test_neg_mean_squared_error,test_r2,experiment
0,0.020303,0.005019,-1645.472550,-0.008680,2
1,0.012604,0.000000,-389.853097,-0.802109,2
2,0.014751,0.003862,-855.686518,-0.187914,2
3,0.005354,0.010468,-7241.115428,-0.005042,2
4,0.015637,0.004622,-720.043887,-0.226430,2


In [ ]:
res_simple_b.mean()

In [39]:
res_simple_dict_c = cross_validate(pipeline_c, x_train, y_train, cv = 5, scoring = scoring)
res_simple_c = pd.DataFrame(res_simple_dict_c).assign(experiment = 3)
res_simple_c

,fit_time,score_time,test_neg_mean_squared_error,test_r2,experiment
0,0.166220,0.004019,-2.972464e+04,-1.722131e+01,3
1,0.077724,0.004802,-2.225579e+25,-1.028781e+23,3
2,0.096250,0.000000,-4.049962e+26,-5.622396e+23,3
3,0.078000,0.000000,-4.263299e+04,-4.917314e+00,3
4,0.068087,0.000000,-1.754474e+04,-2.888344e+01,3


In [ ]:
res_simple_c.mean()

In [40]:
res_simple_dict_d = cross_validate(pipeline_d, x_train, y_train, cv = 5, scoring = scoring)
res_simple_d = pd.DataFrame(res_simple_dict_d).assign(experiment = 2)
res_simple_d

,fit_time,score_time,test_neg_mean_squared_error,test_r2,experiment
0,0.112424,0.005547,-1.932316e+26,-1.184516e+23,2
1,0.091837,0.003846,-4.820088e+04,-2.218101e+02,2
2,0.074998,0.019311,-9.831375e+27,-1.364849e+25,2
3,0.067410,0.013358,-2.293013e+26,-3.182624e+22,2
4,0.083344,0.000000,-1.555181e+27,-2.648895e+24,2


In [ ]:
res_simple_d.mean()

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [103]:
# Define the parameter grid for the LinearRegression step
param_grid_a = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False],
    'regressor__n_jobs': [None, 1, 2, 4, 8],
    'regressor__positive': [True, False]
}

In [104]:
# Perform GridSearchCV on pipeline_a
grid_cv_a = GridSearchCV(
    estimator=pipeline_a, 
    param_grid=param_grid_a, 
    scoring=scoring, 
    cv=5,
    refit='neg_mean_squared_error'
)

In [105]:
# Fit the model
grid_cv_a.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numeric_transfomer',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', LinearRegression())]),
             param_grid={'regressor__copy_X': [True, False],
                         'regressor__fit_intercept': [True, False],
                         'regressor__n_jobs': [None, 1, 2, 4, 8],
                         'regressor__positive': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'r2'])

In [106]:
res = grid_cv_a.cv_results_
res = pd.DataFrame(res)

In [100]:
param_grid_b = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False],
    'regressor__n_jobs': [None, 1, 2, 4, 8],
    'regressor__positive': [True, False]
}

In [101]:
# Perform GridSearchCV on pipeline_b
grid_cv_b = GridSearchCV(
    estimator=pipeline_b, 
    param_grid=param_grid_b, 
    scoring=scoring, 
    cv=5,
    refit='neg_mean_squared_error'
)

In [102]:
# Fit the model
grid_cv_b.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('transform',
                                                                         PowerTransformer(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', LinearRegression())]),
             param_grid={'regressor__copy_X': [True, False],
                         'regressor__fit_intercept': [True, False],
                         'regressor__n_jobs': [None, 1, 2, 4, 8],
                         'regressor__positive': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'r2'])

In [54]:
res = grid_cv_b.cv_results_
res = pd.DataFrame(res)

In [69]:
param_grid_c = {
    'linear__fit_intercept': [True, False],
    'linear__copy_X': [True, False],
    'linear__n_jobs': [None, 1, 2, 4, 8],
    'linear__positive': [True, False]
}

In [70]:
# Perform GridSearchCV on pipeline_c
grid_cv_c = GridSearchCV(
    estimator=pipeline_c, 
    param_grid=param_grid_c, 
    scoring=scoring, 
    cv=5,
    refit='neg_mean_squared_error'
)

In [112]:
# Fit the model
grid_cv_c.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('numeric_transfomer',
                                                                         StandardScaler(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('poly', PolynomialFeatures()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__copy_X': [True, False],
                         'linear__fit_intercept': [True, False],
                         'linear__n_jobs': [None, 1, 2, 4, 8],
                         'linear__positive': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'r2'])

In [113]:
res = grid_cv_c.cv_results_
res = pd.DataFrame(res)

In [114]:
param_grid_d = {
    'linear__fit_intercept': [True, False],
    'linear__copy_X': [True, False],
    'linear__n_jobs': [None, 1, 2, 4, 8],
    'linear__positive': [True, False]
}

In [115]:
# Perform GridSearchCV on pipeline_d
grid_cv_d = GridSearchCV(
    estimator=pipeline_d, 
    param_grid=param_grid_d, 
    scoring=scoring, 
    cv=5,
    refit='neg_mean_squared_error'
)

In [116]:
# Fit the model
grid_cv_d.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('transform',
                                                                         PowerTransformer(),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('poly', PolynomialFeatures()),
                                       ('linear', LinearRegression())]),
             param_grid={'linear__copy_X': [True, False],
                         'linear__fit_intercept': [True, False],
                         'linear__n_jobs': [None, 1, 2, 4, 8],
                         'linear__positive': [True, False]},
             refit='neg_mean_squared_error',
             scoring=['neg_mean_squared_error', 'r2'])

In [117]:
res = grid_cv_d.cv_results_
res = pd.DataFrame(res)

# Evaluate

+ Which model has the best performance?

In [135]:
#best model pipleline_a
best_model_a = grid_cv_a.best_estimator_
grid_cv_a.best_params_

{'regressor__copy_X': True,
 'regressor__fit_intercept': False,
 'regressor__n_jobs': None,
 'regressor__positive': True}

In [124]:
#best model pipleline_b
best_model_b = grid_cv_b.best_estimator_
grid_cv_b.best_params_

{'regressor__copy_X': True,
 'regressor__fit_intercept': False,
 'regressor__n_jobs': None,
 'regressor__positive': True}

In [125]:
#best model pipleline_c
best_model_c = grid_cv_c.best_estimator_
grid_cv_c.best_params_

{'linear__copy_X': True,
 'linear__fit_intercept': False,
 'linear__n_jobs': None,
 'linear__positive': True}

In [126]:
#best model pipleline_d
best_model_d = grid_cv_d.best_estimator_
grid_cv_d.best_params_

{'linear__copy_X': True,
 'linear__fit_intercept': False,
 'linear__n_jobs': None,
 'linear__positive': False}

# Export

+ Save the best performing model to a pickle file.

In [136]:
# Save the best model to a pickle file
with open('best_model_a.pkl', 'wb') as f:
    pickle.dump(best_model_a, f)

In [138]:
# Save the best model to a pickle file
with open('best_model_b.pkl', 'wb') as f:
    pickle.dump(best_model_b, f)

In [139]:
# Save the best model to a pickle file
with open('best_model_c.pkl', 'wb') as f:
    pickle.dump(best_model_c, f)

In [140]:
# Save the best model to a pickle file
with open('best_model_d.pkl', 'wb') as f:
    pickle.dump(best_model_d, f)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [143]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [144]:
import shap

c:\Users\KhanFai1\AppData\Local\miniconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [147]:
import shap
data_transform = pipeline_a.named_steps['preproc_1'].transform(x_test)

explainer = shap.explainers.Linear(
    pipeline_a.named_steps['clf'], 
    data_transform,
    feature_names = pipeline_a.named_steps['preproc_1'].get_feature_names_out())

shap_values = explainer(data_transform)

KeyError: 'preproc_1'

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.